In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.cloud import bigquery
import time
%env GOOGLE_APPLICATION_CREDENTIALS=../secrets/bigquery-service-account.json 
client = bigquery.Client()
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

env: GOOGLE_APPLICATION_CREDENTIALS=../secrets/bigquery-service-account.json


In [3]:
%load_ext google.cloud.bigquery

In [4]:
%%bigquery metadata --project masterarbeit-245718 --verbose 
SELECT * FROM `bigquery-public-data`.crypto_ethereum.INFORMATION_SCHEMA.COLUMNS where table_name = "traces"

Executing query with job ID: 0147291b-6eba-46d6-a315-4aebce4533af
Query executing: 3.87s
Query complete after 4.58s


In [5]:
size_sample_data = 200

### Generate sample data for "call_type"

In [6]:
# %%bigquery call_types_sql_result --project masterarbeit-245718 --verbose 
# select DISTINCT call_type from `bigquery-public-data.crypto_ethereum.traces`
# where DATE(block_timestamp) >= '2019-07-6' AND DATE(block_timestamp) <= '2019-7-7'

In [7]:
call_types = ['call', 'delegatecall', 'staticcall', 'callcode', None]
prob_call_types = [0.7, 0.05, 0.025, 0.025, 0.2]

In [8]:
call_type_sample = np.random.choice(call_types, 2*size_sample_data, p=prob_call_types)

# Generate sample data for "trace_type"

In [9]:
additional_trace_types = ["create", "suicide", "reward", "genesis", "daofork"]
prob_additional_trace_types = [0.025, 0.025, 0.9, 0.025, 0.025]

trace_type_sample = []
for ct in call_type_sample:
    if ct == None:
        trace_type_sample.append(np.random.choice(additional_trace_types, p=prob_additional_trace_types))
    else:
        trace_type_sample.append("call")

### Generate sample data for "status"

In [10]:
# %%bigquery status_values --project masterarbeit-245718 --verbose 
# select DISTINCT status from `bigquery-public-data.crypto_ethereum.traces`
# where DATE(block_timestamp) >= '2019-07-6' AND DATE(block_timestamp) <= '2019-7-7'

In [11]:
status_values = [0, 1]
probs_status_values = [0.05, 0.95]

In [12]:
probs_status_values = [0.05, 0.95]

In [13]:
status_sample = np.random.choice(status_values, 2*size_sample_data, p=probs_status_values)

### Generate sample accounts 

In [14]:
exchanges = ["exchange_{}".format(i) for i in range(1,int(0.05*size_sample_data + 1))]

In [15]:
speculators = ["speculator_{}".format(i) for i in range(1,int(0.95*size_sample_data + 1))]

In [16]:
addresses_testdata = []
addresses_testdata.extend(speculators)
addresses_testdata.extend(exchanges)

In [17]:
addresses_testdata = pd.DataFrame(addresses_testdata, columns=["address"])

## Upload 'addresses' table to bigquery 

In [18]:
addresses_testdata.to_gbq('ethereum_us.addresses_testdata', if_exists="replace")

1it [00:03,  3.73s/it]


### Generate sample transactions (speculators to exchanges)

In [19]:
from_spec_addresses = [np.random.choice(speculators) for i in range(int(2*size_sample_data/2))]
to_ex_addresses = [np.random.choice(exchanges) for i in range(int(2*size_sample_data/2))]
values_spec_to_ex = np.random.randint(1, 20, int(2*size_sample_data/2))

In [20]:
txdata1 = pd.DataFrame(zip(from_spec_addresses, to_ex_addresses, values_spec_to_ex), columns=["from_address", "to_address", "value"])

### Generate sample transactions (exchanges to speculators)

In [21]:
to_spec_addresses = [np.random.choice(speculators) for i in range(int(2*size_sample_data/2))]
from_ex_addresses = [np.random.choice(exchanges) for i in range(int(2*size_sample_data/2))]
values_ex_to_spec = np.random.randint(1, 5, int(2*size_sample_data/2))

In [22]:
txdata2 = pd.DataFrame(zip(from_ex_addresses, to_spec_addresses, values_ex_to_spec), columns=["from_address", "to_address", "value"])

Anmerkung: Die speculators schicken mehr Geld zu den Börsen als umgekehrt.

In [23]:
txdata = txdata1.append(txdata2).reset_index(drop=True)

### Generate sample "block_timestamps"

In [24]:
import datetime as datetime

base = datetime.datetime.utcnow()
block_timestamps = [base - datetime.timedelta(seconds=x) for x in range(0, 12*2*size_sample_data, 12)]

### Merge data to sample "traces" table

In [25]:
traces_sampleData = txdata.copy()

In [26]:
traces_sampleData["status"] = status_sample 

In [27]:
traces_sampleData["call_type"] = call_type_sample 

In [28]:
traces_sampleData["trace_type"] = trace_type_sample 

In [29]:
traces_sampleData["block_timestamp"] = block_timestamps 

In [30]:
traces_sampleData

,from_address,to_address,value,status,call_type,trace_type,block_timestamp
0,speculator_7,exchange_10,3,1,call,call,2020-02-08 13:33:49.942721
1,speculator_121,exchange_6,11,1,call,call,2020-02-08 13:33:37.942721
2,speculator_93,exchange_6,8,1,call,call,2020-02-08 13:33:25.942721
3,speculator_13,exchange_5,2,1,call,call,2020-02-08 13:33:13.942721
4,speculator_138,exchange_6,17,1,call,call,2020-02-08 13:33:01.942721
5,speculator_163,exchange_4,16,1,call,call,2020-02-08 13:32:49.942721
6,speculator_14,exchange_10,3,1,call,call,2020-02-08 13:32:37.942721
7,speculator_136,exchange_1,9,1,call,call,2020-02-08 13:32:25.942721
8,speculator_48,exchange_4,11,1,call,call,2020-02-08 13:32:13.942721
9,speculator_142,exchange_3,7,1,call,call,2020-02-08 13:32:01.942721


### Upload "traces" table to bigquery

In [31]:
traces_sampleData.to_gbq('ethereum_us.traces_sampleData', if_exists="replace")

1it [00:06,  6.94s/it]


### Test SQL command for retrieving "weiReceived", "weiSent"

In [32]:
%%bigquery res1 --project masterarbeit-245718 --verbose 
with weiView as (

  with weiReceivedView as (
        
      -- debits
      select to_address, sum(ifnull(value, 0)) as weiReceived
      from `ethereum_us.traces_sampleData` 
      where to_address is not null
      and status = 1
      and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null) 
      group by to_address
        
  ), weiSentView as (
  
      -- credits
      select from_address, sum(ifnull(value, 0)) as weiSent
      from  `ethereum_us.traces_sampleData` 
      where from_address is not null
      and status = 1
      and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null) 
      group by from_address
  ) 
  select 
  CASE 
    when to_address is not null then to_address
    when from_address is not null then from_address
  end as address, 
  ifnull(weiReceived,0) as weiReceived, 
  ifnull(weiSent,0) as weiSent
  from weiReceivedView full outer join weiSentView on from_address = to_address
) 
select address, weiReceived, weiSent from weiView right join `ethereum_us.addresses_testdata`  using(address)

Executing query with job ID: 6da3e094-ff34-4563-a8c2-7b4a748510b2
Query executing: 1.20s
Query complete after 1.83s


In [33]:
res2 = res1.copy()
res2["balance"] = res1.weiReceived - res1.weiSent
res2 = res2.set_index("address")
res2 = res2.sort_values(by="address", ascending=False)
res2 = res2.astype("float")
res2 = res2.fillna(0.)
balance_result_sql = res2

In [34]:
%%bigquery traces_sampleData --project masterarbeit-245718 --verbose 
select * from `ethereum_us.traces_sampleData`

Executing query with job ID: a58ecfda-fc5e-43b2-b097-6d0439172c2d
Query executing: 0.68s
Query complete after 1.49s


In [35]:
data1 = [row for (index, row) in traces_sampleData.iterrows() if (row.call_type not in ['delegatecall', 'callcode', 'staticcall'] or row.call_type == None) and row.status == 1]
data1 = pd.DataFrame(data1)

In [36]:
data2 = pd.DataFrame(index=set(data1["from_address"].unique()) | set(data1["to_address"].unique()))
data2["weiReceived"] = data1.groupby('to_address').sum().value
data2["weiSent"] = data1.groupby('from_address').sum().value
data2["weiSent"] = data2["weiSent"].fillna(0.)
data2["weiReceived"] = data2["weiReceived"].fillna(0.)
data2["balance"] = data2["weiReceived"] - data2["weiSent"]
data2.index = data2.index.rename("address")
data2 = data2.reindex(addresses_testdata["address"], fill_value=0.)
data2 = data2.sort_values(by="address", ascending=False)
balance_result_py = data2

In [37]:
balance_result_py.head()
balance_result_sql.head()

,weiReceived,weiSent,balance
address,,,
speculator_99,5.0,19.0,-14.0
speculator_98,0.0,0.0,0.0
speculator_97,6.0,0.0,6.0
speculator_96,0.0,0.0,0.0
speculator_95,7.0,0.0,7.0


,weiReceived,weiSent,balance
address,,,
speculator_99,5.0,19.0,-14.0
speculator_98,0.0,0.0,0.0
speculator_97,6.0,0.0,6.0
speculator_96,0.0,0.0,0.0
speculator_95,7.0,0.0,7.0


In [38]:
pd.testing.assert_frame_equal(balance_result_py, balance_result_sql)
print("weiSent, weiReceived Test succeeded!!")

weiSent, weiReceived Test succeeded!!


### Test SQL command for retrieving "txSent" and "txReceived"

In [39]:
%%bigquery res2 --project masterarbeit-245718 --verbose 
with txView as (

  with txSent as (
      SELECT from_address, count(*) as numberOfTranscationsSent FROM `masterarbeit-245718.ethereum_us.traces_sampleData` 
      where to_address is not null and status = 1 and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null) 
      group by from_address
    ), txReceived as (
      SELECT to_address, count(*) as numberOfTranscationsReceived FROM `masterarbeit-245718.ethereum_us.traces_sampleData` 
      where to_address is not null and status = 1 and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null) 
      group by to_address
    ) 
    SELECT 
    CASE  
      WHEN to_address IS NOT NULL THEN to_address
      WHEN from_address IS NOT NULL THEN from_address
    END AS address,
    IFNULL(numberOfTranscationsReceived, 0) as numberOfTranscationsReceived, 
    IFNULL(numberOfTranscationsSent, 0) as numberOfTranscationsSent
    from txReceived FULL OUTER JOIN txSent on to_address = from_address
) 

    select address, numberOfTranscationsReceived, numberOfTranscationsSent from txView right join `ethereum_us.addresses_testdata` using(address)

Executing query with job ID: ac9cd677-82b2-4a63-b51b-2eb9e8693763
Query executing: 0.64s
Query complete after 2.39s


In [40]:
res3 = res2.copy()
res3 = res3.fillna(0.)
res3 = res3.sort_values(by="address", ascending=False)
tx_sent_received_result_sql = res3.set_index("address", drop=True)

In [41]:
data3 = pd.DataFrame(index=set(data1["from_address"].unique()) | set(data1["to_address"].unique()))
data3["numberOfTranscationsReceived"] = data1.groupby('to_address').count().value
data3["numberOfTranscationsSent"] = data1.groupby('from_address').count().value
data3 = data3.fillna(0)
data3 = data3.astype("int")
data3.index = data3.index.rename("address")
data3 = data3.reindex(addresses_testdata["address"], fill_value=0.)
data3 = data3.sort_values(by="address", ascending=False)
tx_sent_received_result_py = data3

In [42]:
pd.testing.assert_frame_equal(tx_sent_received_result_py, tx_sent_received_result_sql)
print("txSent, txReceived Test succeeded !!")

txSent, txReceived Test succeeded !!


### Test avg time between tx

In [43]:
%%bigquery res4 --project masterarbeit-245718 --verbose 
with timeRecView as (

  with receivedTx as (
    SELECT to_address, count(*) as numberOfTranscationsReceived FROM `masterarbeit-245718.ethereum_us.traces_sampleData` 
    where to_address is not null 
      and status = 1 
      and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null)
    group by to_address),
  
  timeStampDiffs as (
    SELECT to_address, TIMESTAMP_DIFF(MAX(block_timestamp), MIN( block_timestamp ), second ) as timestampDiff
    FROM `masterarbeit-245718.ethereum_us.traces_sampleData`
    where to_address is not null 
      and status = 1 
      and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null)
    group by to_address
  
  ) select to_address as address, 
  CASE 
    when (numberOfTranscationsReceived - 1)  > 0 then timestampDiff / (numberOfTranscationsReceived - 1) 
    else 0
  end as avgTimeDiffBetweenReceivedTransactions
  from receivedTx inner join  timeStampDiffs using(to_address)
)

select address, ifnull(avgTimeDiffBetweenReceivedTransactions,0) as avgTimeDiffBetweenReceivedTransactions from timeRecView right join `ethereum_us.addresses_testdata` using(address)

Executing query with job ID: 2c6ce4d3-61f0-4e21-8dc8-2413cfc325b7
Query executing: 1.24s
Query complete after 1.94s


In [44]:
res6 = res4.set_index("address", drop=True)
res6 = res6.fillna(0.)
res6 = res6.sort_values(by="address")
avg_time_diff_receivedtx_result_sql = res6

In [45]:
%%bigquery res5 --project masterarbeit-245718 --verbose 
with timeSentView as (

  with sentTx as (
    SELECT from_address, count(*) as numberOfTranscationsSent FROM `masterarbeit-245718.ethereum_us.traces_sampleData` 
    where to_address is not null 
      and status = 1 
      and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null)
    group by from_address),
  timeStampDiffs as (
    SELECT from_address, TIMESTAMP_DIFF(MAX(block_timestamp), MIN( block_timestamp ), second ) as timestampDiff
    FROM `masterarbeit-245718.ethereum_us.traces_sampleData`
    where to_address is not null 
      and status = 1 
      and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null)
    group by from_address
  ) select from_address as address, 
  CASE 
    when (numberOfTranscationsSent - 1)  > 0 then timestampDiff / (numberOfTranscationsSent - 1) 
    else 0
  end as avgTimeDiffBetweenSentTransactions
     from sentTx inner join  timeStampDiffs using(from_address)
)

select address, ifnull(avgTimeDiffBetweenSentTransactions,0) as avgTimeDiffBetweenSentTransactions from timeSentView right join `ethereum_us.addresses_testdata` using(address)

Executing query with job ID: 74cd059a-7aa8-48c2-b831-7dd79fb6e9b0
Query executing: 1.17s
Query complete after 2.09s


In [46]:
res6 = res5.set_index("address", drop=True)
res6 = res6.fillna(0.)
res6 = res6.sort_values(by="address")
avg_time_diff_senttx_result_sql = res6

In [47]:
res7 = [row for (index, row) in traces_sampleData.iterrows() if (row.call_type not in ['delegatecall', 'callcode', 'staticcall'] or row.call_type == None) and row.status == 1]
res7 = pd.DataFrame(res7)
res8 = res7.groupby("from_address").max().block_timestamp
res9 = res7.groupby("from_address").min().block_timestamp
res10 = res8 - res9
res10 = res10.rename("seconds_diff")
res10 = tx_sent_received_result_py.join(res10, how="right").drop("numberOfTranscationsReceived", axis=1)
res10 = res10.fillna(0.)
res10["avgTimeDiffBetweenSentTransactions"] = res10["seconds_diff"] / (res10["numberOfTranscationsSent"] - 1)
res10["avgTimeDiffBetweenSentTransactions"] = res10["avgTimeDiffBetweenSentTransactions"].fillna(datetime.timedelta(0))
res10 = res10.reindex(addresses_testdata["address"], fill_value=0.)
res10.index = res10.index.rename("address")
res10 = res10.sort_values(by="address")
res10 = res10.drop(["numberOfTranscationsSent", "seconds_diff"],axis=1)
res10["avgTimeDiffBetweenSentTransactions"] = [ts.total_seconds() for ts in res10["avgTimeDiffBetweenSentTransactions"]]
avg_time_diff_senttx_result_py = res10

/home/mgsgde/.local/lib/python3.6/site-packages/pandas/core/arrays/timedeltas.py:523: RuntimeWarning: invalid value encountered in true_divide
  result = self._data / other


In [48]:
pd.testing.assert_frame_equal(avg_time_diff_senttx_result_sql, avg_time_diff_senttx_result_py)
print("avgTimeDiffBetweenSentTransactions Test succeeded !!")

avgTimeDiffBetweenSentTransactions Test succeeded !!


In [49]:
features = balance_result_sql.join(tx_sent_received_result_sql,how="left")
features = features.join(avg_time_diff_senttx_result_sql)
features = features.join(avg_time_diff_receivedtx_result_sql)
features = features.sort_values(by="balance", ascending=False)
features = features.fillna(0.0)
import sys
# addresses that have sent/received only one transaction get the avg time max * 2
features["avgTimeDiffBetweenSentTransactions"] = features["avgTimeDiffBetweenSentTransactions"].replace(to_replace=0.0, value=2 * max(features["avgTimeDiffBetweenSentTransactions"]))
features["avgTimeDiffBetweenReceivedTransactions"] = features["avgTimeDiffBetweenReceivedTransactions"].replace(to_replace=0.0, value=2 * max(features["avgTimeDiffBetweenReceivedTransactions"]))

In [50]:
features = features.reset_index()

In [51]:
features.to_gbq('ethereum_us.features_sampleData', if_exists="replace")

1it [00:07,  7.08s/it]


# Test mined Blocks

In [52]:
%%bigquery sql_res_minedblocks --project masterarbeit-245718 --verbose 

with minedBlocksView as (
    SELECT to_address as address, count(*) as mined_blocks FROM `masterarbeit-245718.ethereum_us.traces_sampleData`
        where trace_type = "reward"
        group by to_address
    )
select address, ifnull(mined_blocks,0) as minedBlocks from minedBlocksView right join `ethereum_us.addresses_testdata` using(address)


Executing query with job ID: cdeffaa7-3ae8-4289-9c66-f112da63d4b1
Query executing: 0.79s
Query complete after 1.72s


In [53]:
features.head()

,address,weiReceived,weiSent,balance,numberOfTranscationsReceived,numberOfTranscationsSent,avgTimeDiffBetweenSentTransactions,avgTimeDiffBetweenReceivedTransactions
0,exchange_1,243.0,34.0,209.0,21.0,15.0,168.857143,108.000000
1,exchange_4,215.0,27.0,188.0,22.0,11.0,202.800000,110.285714
2,exchange_6,223.0,44.0,179.0,22.0,16.0,139.200000,98.285714
3,exchange_3,205.0,40.0,165.0,16.0,18.0,123.529412,149.600000
4,exchange_2,179.0,41.0,138.0,15.0,17.0,130.500000,145.714286


# Test var(avgTimeDiff...)

In [221]:
%%bigquery res_sql_stddev --project masterarbeit-245718 --verbose 

with timestamps_diffs as (
    
    with timestamps_preceding_tx as (
        
        with timestamps_sent_tx as (
            select from_address, block_timestamp from `masterarbeit-245718.ethereum_us.traces_sampleData`
                where from_address is not null 
                  and status = 1 
                  and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null)
        )
        
        select from_address, block_timestamp,
            lag(block_timestamp) OVER (partition by from_address order by block_timestamp asc) as preceding_block_timestamp 
        from timestamps_sent_tx
    )
    
    select from_address, block_timestamp, preceding_block_timestamp, 
        TIMESTAMP_DIFF(block_timestamp, preceding_block_timestamp, second) as timestampdiff
    from timestamps_preceding_tx
)
select from_address, STDDEV_SAMP(timestampdiff) as stddev_sample  
from timestamps_diffs group by from_address

Executing query with job ID: e8e444c2-8651-4c2d-b3c5-cb6de7a2e186
Query executing: 0.69s
Query complete after 1.30s


In [222]:
stddev_result_sql = res_sql_stddev.set_index("from_address")
stddev_result_sql = stddev_result_sql.reindex(addresses_testdata["address"])
stddev_result_sql = stddev_result_sql.fillna(res_sql_stddev["stddev_sample"].max())
stddev_result_sql = stddev_result_sql.sort_values(by="address")

,stddev_sample
address,
exchange_1,172.711791
exchange_10,102.068884
exchange_2,132.896953
exchange_3,142.161052
exchange_4,141.923923
exchange_5,98.267342
exchange_6,109.023720
exchange_7,185.803502
exchange_8,211.673849


In [223]:
%%bigquery res_sql_stddev --project masterarbeit-245718 --verbose 

with timestamps_diffs as (
    
    with timestamps_preceding_tx as (
        
        with timestamps_received_tx as (
            select to_address, block_timestamp from `masterarbeit-245718.ethereum_us.traces_sampleData`
                where to_address is not null 
                  and status = 1 
                  and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null)
        )
        
        select to_address, block_timestamp,
            lag(block_timestamp) OVER (partition by to_address order by block_timestamp asc) as preceding_block_timestamp 
        from timestamps_received_tx
    )
    
    select to_address, block_timestamp, preceding_block_timestamp, 
        TIMESTAMP_DIFF(block_timestamp, preceding_block_timestamp, second) as timestampdiff
    from timestamps_preceding_tx
)

select to_address, STDDEV_SAMP(timestampdiff) as stddev_sample  
from timestamps_diffs group by to_address

Executing query with job ID: b85f0c29-53d7-4389-b7b1-ca51a1c41e67
Query executing: 0.63s
Query complete after 1.35s


In [231]:
stddev_result_sql = res_sql_stddev.set_index("to_address")
stddev_result_sql = stddev_result_sql.reindex(addresses_testdata["address"])
stddev_result_sql = stddev_result_sql.fillna(res_sql_stddev["stddev_sample"].max())
stddev_result_sql = stddev_result_sql.sort_values(by="stddev_sample")

,stddev_sample
address,
speculator_56,59.396970
exchange_6,90.408596
exchange_1,92.379082
exchange_8,100.361699
exchange_9,109.320214
exchange_5,123.562797
exchange_2,129.574112
exchange_3,132.766605
exchange_4,140.231645


# Verifikation der var(avgTimeDiff...) Tabelle via Python

In [234]:
res7 = [row for (index, row) in traces_sampleData.iterrows() if (row.call_type not in ['delegatecall', 'callcode', 'staticcall'] or row.call_type == None) and row.status == 1]
res7 = pd.DataFrame(res7)
res7 = res7.sort_values(by="block_timestamp", ascending=True)

In [235]:
def calculateDifference(df):
#     from_address = df["from_address"].reset_index(drop=True)[0]
    variance = df["block_timestamp"].diff().std()
    return variance
#     return pd.DataFrame({"variance": [variance]})

res8 = res7.groupby("to_address").apply(lambda df: df["block_timestamp"].diff().std() )
res8 = pd.DataFrame(res8, columns=["stddev_sample"])
res8 = res8.reindex(addresses_testdata["address"])
res8 = res8.fillna(res8.max())
res8["stddev_sample"] = res8["stddev_sample"] / np.timedelta64(1, 's')
res8 = res8.sort_values(by="stddev_sample")
stddev_result_py = res8

,stddev_sample
address,
speculator_56,59.396970
exchange_6,90.408596
exchange_1,92.379082
exchange_8,100.361699
exchange_9,109.320214
exchange_5,123.562797
exchange_2,129.574112
exchange_3,132.766605
exchange_4,140.231645


In [239]:
# pd.testing.assert_frame_equal(stddev_result_py, stddev_result_sql)
np.testing.assert_almost_equal(list(stddev_result_py["stddev_sample"]), list(stddev_result_sql["stddev_sample"]), decimal=2)
print("stddev Test succeeded !!")

stddev Test succeeded !!


# Deprecated

In [326]:
%%bigquery res6 --project masterarbeit-245718 --verbose 

SELECT
* 
FROM
`masterarbeit-245718.ethereum_us.INFORMATION_SCHEMA.TABLES`


Executing query with job ID: 5036d0d3-7765-405c-8eff-954e26a3cf6c
Query executing: 0.62s
Query complete after 1.40s


In [327]:
tablesToDelete = [tn for tn in res6["table_name"] if tn.startswith("top40k_19_20")]

for t in tablesToDelete:
     client.delete_table("ethereum_us.{}".format(t), not_found_ok=True)  